# Localで実行

In [1]:
pip install azureml-explain-model

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azureml-interpret

In [4]:
# Decision Tree Classifier
# Import libraries
import pandas as pd

# Read dataset
df = pd.read_csv('./data/adultincome trunc.csv')

# Create Dummy variables
data_prep = pd.get_dummies(df, drop_first=True)

# Create X and Y Variables
X = data_prep.iloc[:, :-1]
Y = data_prep.iloc[:, -1]


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size = 0.3, random_state = 1234, stratify=Y)

# Import and train Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1234)

trained_model = rfc.fit(X_train, Y_train)

# Test the RFC model
Y_predict = rfc.predict(X_test)

# Evaluate the RFC model
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, Y_predict)
score = rfc.score(X_test, Y_test)

# Create explanations for the model
# Import the Tabular Explainer class
from interpret.ext.blackbox import TabularExplainer

# Define the Classes and features
# 0と1に名前を割り振る
classes  = ['Not Greater than 50K', 'Greater than 50K']
features = list(X.columns)

# Create the Tabular Explainer object
tab_explainer = TabularExplainer(trained_model,
                                 X_train,
                                 features=features,
                                 classes=classes)

# Get the global explanations
global_explanation = tab_explainer.explain_global(X_train)

# Get the feature importance data
global_fi = global_explanation.get_feature_importance_dict()

Could not import lightgbm, required if using LGBMExplainableModel


In [7]:
# Print the global feature importance values
for feature, importance in global_fi.items():
    print(feature,":", importance)

marital status_Married : 0.134457897553098
age : 0.08737999026945
hours per week : 0.054862878214208904
education_ HS-grad : 0.05459228226812601
marital status_ Never-married : 0.04201492036510847
education_ Some-college : 0.03902094175894924
gender_ Male : 0.026954924512638787
education_ Masters : 0.013130510698434426
race_ White : 0.013018742998495862
wc_ Private : 0.008260686989919535
wc_ Local-gov : 0.006514623134033063
education_ Prof-school : 0.006411507490632027
race_ Black : 0.00431141694330449
education_ Doctorate : 0.003320328208831908
race_ Asian-Pac-Islander : 0.0021244991478086886
marital status_ Widowed : 0.0016788032177812158
race_ Other : 0.0003622585777075659
education_ Preschool : 3.30258572723802e-05


In [8]:
# Get Local feature importances
X_explain = X_test[0:5]

# Create Local explanations object
local_explanation = tab_explainer.explain_local(X_explain)

# Extract the feature names and corresponding importance values
local_features   = local_explanation.get_ranked_local_names()
local_importance = local_explanation.get_ranked_local_values()

In [31]:
classes

['Not Greater than 50K', 'Greater than 50K']

In [35]:
# 例えば1つ目のデータはeducation_HS-gradがラベル0に対して0.157程影響していることが分かる
df = pd.DataFrame(local_features)
df

,0,1,2,3,4
0,"[education_ HS-grad, hours per week, wc_ Priva...","[marital status_Married, hours per week, educa...","[marital status_Married, education_ HS-grad, g...","[wc_ Local-gov, education_ Prof-school, educat...","[education_ Some-college, age, hours per week,..."
1,"[marital status_Married, age, marital status_ ...","[marital status_ Never-married, education_ Som...","[age, marital status_ Never-married, education...","[marital status_Married, education_ Masters, h...","[marital status_Married, marital status_ Never..."


In [36]:
df = pd.DataFrame(local_importance)
df

,0,1,2,3,4
0,"[0.15753368669907972, 0.06271450180562069, 0.0...","[0.11368978878763233, 0.05826405235652043, 0.0...","[0.1342190342125361, 0.042854307978928134, 0.0...","[0.03963626880286255, 0.002164326904273668, 0....","[0.11024063654033374, 0.10225658708107799, 0.0..."
1,"[0.1499128064951366, 0.06463253768495787, 0.04...","[0.01719082246795429, 0.011580996830395821, 0....","[0.0620378807796918, 0.04120158213742204, 0.02...","[0.14615858819831004, 0.11329883795157034, 0.0...","[0.11108643974390256, 0.02889560997008754, 0.0..."


In [38]:
# 整理して表示
for i in range(0, len(local_features)):
    labels = local_features[i]
    print('\n Feature support value for : ', classes[i])
    
    for j in range(0, len(labels)):
        
        # 判定したラベルと一致する重要度を確認する
        if Y_predict[j] ==i:
            print('\n\tObservation number : ', j+1)
            feature_names = labels[j]
            
            print('\t\t', 'Feature Name'.ljust(30), " Value")
            print('\t\t', "-"*30, "-"*10)
            
            for k in range(0, len(feature_names)):
                print("\t\t", feature_names[k].ljust(30), round(local_importance[i][j][k], 6))


 Feature support value for :  Not Greater than 50K

	Observation number :  1
		 Feature Name                    Value
		 ------------------------------ ----------
		 education_ HS-grad             0.157534
		 hours per week                 0.062715
		 wc_ Private                    0.0179
		 education_ Masters             0.012778
		 education_ Prof-school         0.004132
		 race_ Black                    0.003311
		 education_ Doctorate           0.001875
		 education_ Preschool           -1.1e-05
		 race_ Other                    -0.000325
		 wc_ Local-gov                  -0.000783
		 race_ Asian-Pac-Islander       -0.001856
		 marital status_ Widowed        -0.002629
		 gender_ Male                   -0.006227
		 race_ White                    -0.013401
		 education_ Some-college        -0.022706
		 marital status_ Never-married  -0.04377
		 age                            -0.064633
		 marital status_Married         -0.149913

	Observation number :  2
		 Feature Name              

In [39]:
# 例えば上で、Greater than 50Kと判定した No.4が marital_status_Married = 1となっており、
# 結婚していることが収入が高いと判断されることにつながったことが分かる
# 逆にマイナスのものは、収入が低いと判断される方向に影響している
X_explain

,age,hours per week,wc_ Local-gov,wc_ Private,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital status_ Never-married,marital status_ Widowed,marital status_Married,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,gender_ Male
347,51,40,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1
630,34,30,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
598,47,40,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1656,43,60,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1
1192,32,40,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,1


# Remote Computeで実行

In [44]:
# Run an Experiment in an Azureml environment and 
# Import the Azure ML classes
from azureml.core import Workspace, Experiment

# Access the workspace using config.json
print("Accessing the workspace from job....")
ws = Workspace.from_config("./config")

# Get the input dataset
print("Accessing the Adult Income dataset...")
input_ds = ws.datasets.get('AdultIncome')

# Create custom environment
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

# Create the environment
myenv = Environment(name="MyEnvironment")

# Create the dependencies object
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip', 'pandas'],
                                     pip_packages=['azureml-defaults', 'azureml-interpret'])

myenv.python.conda_dependencies = myenv_dep

# Register the environment
print("Registering the environment...")
myenv.register(ws)

# Create the compute Cluster 
# Specify the cluster name
cluster_name = "my-cluster-001"

# Provisioning configuration using AmlCompute
from azureml.core.compute import AmlCompute

print("Accessing the compute cluster...")

if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(
                                     vm_size="STANDARD_D11_V2",
                                     max_nodes=2)

    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster = ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it...")

# Create a script configuration for custom environment of myenv
from azureml.core import ScriptRunConfig

print("Creating the ScriptRunConfig....")
script_config = ScriptRunConfig(source_directory="./script",
                                script="360 - Model explain script.py",
                                arguments = ['--input-data', input_ds.as_named_input('raw_data')],
                                environment=myenv,
                                compute_target=cluster)

# Create the experiment and run
print("Creating the experiment...")
new_experiment = Experiment(workspace=ws, name='Explainer_Exp001')

print("Submittting the experiment...")
new_run = new_experiment.submit(config=script_config)

new_run.wait_for_completion(show_output=True)

Accessing the workspace from job....
Accessing the Adult Income dataset...
Registering the environment...
Accessing the compute cluster...
my-cluster-001 , compute cluster found. Using it...
Creating the ScriptRunConfig....
Creating the experiment...
Submittting the experiment...
RunId: Explainer_Exp001_1665049068_cd926590
Web View: https://ml.azure.com/runs/Explainer_Exp001_1665049068_cd926590?wsid=/subscriptions/3467f739-a57b-4612-9de8-72a6616c01b3/resourcegroups/AzuremlSDKRG00/workspaces/Azureml-SDK-WS01&tid=bcd8db96-8bb9-4f0d-af35-e471bf92c072

Streaming azureml-logs/20_image_build_log.txt

2022/10/06 09:37:54 Downloading source code...
2022/10/06 09:37:55 Finished downloading source code
2022/10/06 09:37:56 Creating Docker network: acb_default_network, driver: 'bridge'
2022/10/06 09:37:56 Successfully set up Docker network: acb_default_network
2022/10/06 09:37:56 Setting up Docker configuration...
2022/10/06 09:37:56 Successfully set up Docker configuration
2022/10/06 09:37:56 Log


mkl-2021.4.0         | 219.1 MB  | ########## | 100% 

libgfortran-ng-7.5.0 | 22 KB     |            |   0% 
libgfortran-ng-7.5.0 | 22 KB     | ########## | 100% 

libffi-3.3           | 54 KB     |            |   0% 
libffi-3.3           | 54 KB     | ########## | 100% 

mkl-service-2.4.0    | 62 KB     |            |   0% 
mkl-service-2.4.0    | 62 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... 

    Installed package of scikit-learn can be accelerated using scikit-learn-intelex.
    More details are available here: https://intel.github.io/scikit-learn-intelex

    For example:

        $ conda install scikit-learn-intelex
        $ python -m sklearnex my_application.py

    

done
Installing pip dependencies: ...working... 
Ran pip subprocess with arguments:
['/azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/bin/python', '-m', 'pip', 'install', '-U', '-r', '/azureml-environme

Removing intermediate container 02cc38f0cac8
 ---> e8ae2222139d
Step 9/21 : ENV PATH /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/bin:$PATH
 ---> Running in 53eb892b92bc
Removing intermediate container 53eb892b92bc
 ---> f0ceee683028
Step 10/21 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> 18065b3fe861
Step 11/21 : RUN echo "Copying environment context"
 ---> Running in f9ea998e5e89
Copying environment context
Removing intermediate container f9ea998e5e89
 ---> 040558a954f1
Step 12/21 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> fef8a235f8f5
Step 13/21 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7
 ---> Running in 3bf9d07f784d
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with mate


Streaming user_logs/std_log.txt

/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_fe8c64e9144e45ea8ac871bd401f32c7/lib/libtinfo.so.6: no version information availa

  1%|          | 132/22792 [00:30<1:03:38,  5.93it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False)

...

 96%|█████████▌| 21867/22792 [1:28:35<03:52,  3.97it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=Fals

 97%|█████████▋| 22181/22792 [1:29:53<02:37,  3.88it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=Fals

 98%|█████████▊| 22424/22792 [1:30:53<01:18,  4.69it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=Fals

100%|█████████▉| 22770/22792 [1:32:14<00:03,  6.82it/s]The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=Fals

{'runId': 'Explainer_Exp001_1665049068_cd926590',
 'target': 'my-cluster-001',
 'status': 'Completed',
 'startTimeUtc': '2022-10-06T09:50:57.124197Z',
 'endTimeUtc': '2022-10-06T11:24:43.990119Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '8044adf9-6b61-4fbf-b2e9-8893a502268c',
  'azureml.git.repository_uri': 'https://github.com/Nakamukai1/Study-Records.git',
  'mlflow.source.git.repoURL': 'https://github.com/Nakamukai1/Study-Records.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '9cda39952d879c5b84494816ab944186176351fe',
  'mlflow.source.git.commit': '9cda39952d879c5b84494816ab944186176351fe',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'c49e90a9-7b72-4e4e-8e83-c9940d37406e'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'raw_d

In [45]:
# Machine Learning Studioで確認可能
# Explanation(preview) ⇒ Aggregate feature importance ⇒ View dependence plot 
# もしくは Individual feature importance

# Download Explanation

In [51]:
from azureml.interpret import ExplanationClient
from azureml.core import Workspace

ws = Workspace.from_config("./config")
new_run = ws.get_run('Explainer_Exp001_1665049068_cd926590') # IDを記入

explain_client = ExplanationClient.from_run(new_run)

# Tabular Explanerとしてダウンロードできる
downloaded_explanations = explain_client.download_model_explanation()

# 結果の確認
feature_importances = downloaded_explanations.get_feature_importance_dict()

In [52]:
feature_importances

{'ms_ Married-civ-spouse': 0.10361359074737249,
 'edu_num': 0.037364331854658386,
 'cg': 0.032745723057124254,
 'age': 0.030052037428192478,
 'hpw': 0.026701250966546082,
 'fw': 0.01789692988993553,
 'occ_ Exec-managerial': 0.01239302978206919,
 'occ_ Prof-specialty': 0.009029638159730157,
 'gender_ Male': 0.008432491339799252,
 'cl': 0.00791793700833217,
 'ms_ Never-married': 0.007408711386185157,
 'rel_ Not-in-family': 0.006366744183870222,
 'edu_ HS-grad': 0.006036472386382093,
 'edu_ Bachelors': 0.005235170976768415,
 'wc_ Private': 0.004860779372665531,
 'rel_ Wife': 0.004192076304914305,
 'rel_ Own-child': 0.0038010439251881193,
 'edu_ Some-college': 0.0037801497446112407,
 'occ_ Craft-repair': 0.0036568877313043983,
 'occ_ Sales': 0.0033542122340373013,
 'edu_ Masters': 0.003289646551437502,
 'rel_ Unmarried': 0.0027448914387742792,
 'race_ White': 0.0026003897170745003,
 'occ_ Tech-support': 0.002284819012833854,
 'occ_ Other-service': 0.002228825847806704,
 'wc_ Self-emp-inc':